In [2]:
import pandas as pd

from Driver.driver import compute_alpha_with_markov_strat_for_ticker, retrieve_and_add_markov, compute_alpha_with_markov_strat_for_list_tickers, retrieve_data_and_add_features, assign_states_and_calculate_transition_probabilities
from Scrapers.yf_scraper import YFScraper
from Markov_State_Determiners.Czech_State_Determiner import CzechStateDeterminer
from Transition_Probability_Calculation.Empirical_Calculator import EmpiricalCalculator
from Markov_Analytics.State_Determiner_Value_Report import generate_value_report_from_scratch
from Markov_Analytics.Analytics_Util import output_significant_cov
from report_processor import to_cov_matrix
from Transition_Probability_Calculation.Contour_Empirical_Calculator import ContourEmpiricalCalculator
from Basic_Feature_Selection.feature_addition import add_basic_technical_analysis_vars
from Markov_Strategies.Czech_State_Determiner.Czech_Strategy1 import CzechStrategy1
"""
REPORT INDICES:
Avg Alpha
Adj Alpha
M_P
BH_P
Ind Removed
Avg. Markov_R/MU_T
Percentile (Against Random)
Avg. BuyHold_R/MU_T
M/BH DTs
markov_dts
buy_hold_dts
Random Mean
Random Standard Deviation
macro_t_units


For the analytics, the indices are listed as follows:
State Combo
Avg Return %
Profitable %
Over B %
STD
Num Instances
Returns
"""

'\nREPORT INDICES:\nAvg Alpha\nAdj Alpha\nM_P\nBH_P\nInd Removed\nAvg. Markov_R/MU_T\nPercentile (Against Random)\nAvg. BuyHold_R/MU_T\nM/BH DTs\nmarkov_dts\nbuy_hold_dts\nRandom Mean\nRandom Standard Deviation\nmacro_t_units\n\n\nFor the analytics, the indices are listed as follows:\nState Combo\nAvg Return %\nProfitable %\nOver B %\nSTD\nNum Instances\nReturns\n'

In [3]:
"""
k = 20
osc = k // 3
ta_vars_to_czech_out = [f'EMA_{k}', f'RSI_{k}', 'MACD', f'Stochastic_Oscillator_{osc}_{k}', f'CMF_{k}',
                        f'volume_oscillator', 'force_index', 'on_balance_volume', 'aroon_oscillator',
                        'volume_price_trend']
rolling_mean_function = lambda data, col_to_sd: data[col_to_sd].ewm(span=k, adjust=False).mean()
curr_ta_var = 'daily_return'
"""


"\nk = 20\nosc = k // 3\nta_vars_to_czech_out = [f'EMA_{k}', f'RSI_{k}', 'MACD', f'Stochastic_Oscillator_{osc}_{k}', f'CMF_{k}',\n                        f'volume_oscillator', 'force_index', 'on_balance_volume', 'aroon_oscillator',\n                        'volume_price_trend']\nrolling_mean_function = lambda data, col_to_sd: data[col_to_sd].ewm(span=k, adjust=False).mean()\ncurr_ta_var = 'daily_return'\n"

In [6]:
from Markov_Strategy_Determiner.Kelly_Strategy_Determiner import KellyStrategyDeterminer

"""
This cell generates a Czech-Based State-Determiner and Optimal 1D strategy on a number of technical variables.

There are some issues with columns with high values (volume) that we can solve later by standardizing values.
"""

tickers = ['SPY', 'JPM', 'WFC', 'TFC', 'BAC', 'C', 'USB', 'PNC']
tickers = ['SPY', 'JPM', 'WFC']
start = '2010-01-01'
end = '2024-01-01'

scraper = YFScraper()
state_determiner = CzechStateDeterminer(k=1, l=20)
transition_probability_calculator = EmpiricalCalculator(state_determiner.possible_states)
generate_value_report_from_scratch(tickers, state_determiner, scraper, strategy_determiner=KellyStrategyDeterminer, start=start, end=end, transition_probability_calculator=transition_probability_calculator,test_against_random=False, N=100, value_report_prefix='State_Determiner_Value_Reports')


/Users/jeremylevitt/IdeaProjects/Markov_Structured/venv/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed

TCKR: SPY - Generating Value Report on State Space . . .
TCKR: USB - Generating Value Report on State Space . . .



/Users/jeremylevitt/IdeaProjects/Markov_Structured/venv/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
/Users/jeremylevitt/IdeaProjects/Markov_Structured/venv/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


TCKR: C - Generating Value Report on State Space . . .
Assigning states . . .
Calculating transition probabilities . . .
Generating MAX 1D return for SPY
Success with initial guess:
 message: successful convergence
 success: True
     fun: [ 1.120e-07]
       x: 2.9427093567754636
     nit: 2
    nfev: 3
  method: df-sane
The optimal Kelly bet fraction S is: 2.9427093567754636
Success with initial guess:
 message: successful convergence
 success: True
     fun: [ 1.887e-07]
       x: 3.316296790243128
     nit: 2
    nfev: 3
  method: df-sane
The optimal Kelly bet fraction S is: 3.316296790243128
Success with initial guess:
 message: successful convergence
 success: True
     fun: [ 9.016e-08]
       x: 7.279475439720282
     nit: 4
    nfev: 5
  method: df-sane
The optimal Kelly bet fraction S is: 7.279475439720282
Success with initial guess:
 message: successful convergence
 success: True
     fun: [ 1.313e-08]
       x: -1.3914346310278296
     nit: 3
    nfev: 4
  method: df-sane
T

/Users/jeremylevitt/IdeaProjects/Markov_Structured/Kelly_Box/Kelly_Box.py:62: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  result, error = quad(integrand, minimum_return, maximum_return, limit=10000)
/Users/jeremylevitt/IdeaProjects/Markov_Structured/Kelly_Box/Kelly_Box.py:60: RuntimeWarning: divide by zero encountered in divide
  return return_pdf(x) * x / (1 + S * x)
/Users/jeremylevitt/IdeaProjects/Markov_Structured/Kelly_Box/Kelly_Box.py:62: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
  result, error = quad(integrand, minimum_return, maximum_return, limit=10000)
/Users/jeremylevitt/IdeaProjects/Markov_Structured/Kelly_Box/Kelly_Box.py:62: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can 

Success with initial guess:
 message: successful convergence
 success: True
     fun: [ 3.748e-06]
       x: -260671.54167275032
     nit: 20
    nfev: 21
  method: df-sane
The optimal Kelly bet fraction S is: -260671.54167275032
Success with initial guess:
 message: successful convergence
 success: True
     fun: [ 7.870e-07]
       x: 4.377082796478432
     nit: 3
    nfev: 4
  method: df-sane
The optimal Kelly bet fraction S is: 4.377082796478432
Success with initial guess:
 message: successful convergence
 success: True
     fun: [ 1.558e-07]
       x: 9.010978973997934
     nit: 4
    nfev: 5
  method: df-sane
The optimal Kelly bet fraction S is: 9.010978973997934
Beginning backtest . . .


/Users/jeremylevitt/IdeaProjects/Markov_Structured/Driver/driver.py:66: RuntimeWarning: invalid value encountered in scalar power
  avg_m_unit, avg_bh_unit = m_p ** (1 / macro_t_units), bh_p ** (1 / macro_t_units)
/Users/jeremylevitt/IdeaProjects/Markov_Structured/Driver/driver.py:42: RuntimeWarning: invalid value encountered in scalar power
  yrly_adj_markov = (m_p / val) ** (1 / macro_t_units)


Generating report . . .
Generated report!
Assigning states . . .
Calculating transition probabilities . . .
Generating MAX 1D return for C
Success with initial guess:
 message: successful convergence
 success: True
     fun: [ 1.796e-08]
       x: 5.733031661199816
     nit: 4
    nfev: 5
  method: df-sane
The optimal Kelly bet fraction S is: 5.733031661199816
Success with initial guess:
 message: successful convergence
 success: True
     fun: [ 7.687e-11]
       x: 4.093516349552049
     nit: 4
    nfev: 5
  method: df-sane
The optimal Kelly bet fraction S is: 4.093516349552049
Success with initial guess:
 message: successful convergence
 success: True
     fun: [ 2.571e-09]
       x: 3.7186116120415
     nit: 4
    nfev: 5
  method: df-sane
The optimal Kelly bet fraction S is: 3.7186116120415


/Users/jeremylevitt/IdeaProjects/Markov_Structured/Kelly_Box/Kelly_Box.py:60: RuntimeWarning: divide by zero encountered in divide
  return return_pdf(x) * x / (1 + S * x)
/Users/jeremylevitt/IdeaProjects/Markov_Structured/Kelly_Box/Kelly_Box.py:62: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  result, error = quad(integrand, minimum_return, maximum_return, limit=10000)
/Users/jeremylevitt/IdeaProjects/Markov_Structured/Kelly_Box/Kelly_Box.py:62: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
  result, error = quad(integrand, minimum_return, maximum_return, limit=10000)
/Users/jeremylevitt/IdeaProjects/Markov_Structured/Kelly_Box/Kelly_Box.py:62: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  result, error = quad(integrand, minimum_return, maximum_return, limit=10000)
/Users/

Success with initial guess:
 message: successful convergence
 success: True
     fun: [ 1.194e-05]
       x: 83654.5773126099
     nit: 16
    nfev: 17
  method: df-sane
The optimal Kelly bet fraction S is: 83654.5773126099
Success with initial guess:
 message: successful convergence
 success: True
     fun: [ 1.681e-08]
       x: 6.096638976130905
     nit: 4
    nfev: 5
  method: df-sane
The optimal Kelly bet fraction S is: 6.096638976130905
Success with initial guess:
 message: successful convergence
 success: True
     fun: [ 3.770e-07]
       x: 15.585897799944775
     nit: 6
    nfev: 7
  method: df-sane
The optimal Kelly bet fraction S is: 15.585897799944775
Success with initial guess:
 message: successful convergence
 success: True
     fun: [ 1.576e-09]
       x: 4.485968488910717
     nit: 4
    nfev: 5
  method: df-sane
The optimal Kelly bet fraction S is: 4.485968488910717
Success with initial guess:
 message: successful convergence
 success: True
     fun: [ 6.016e-07]
   

In [5]:
values = output_significant_cov(cov_matrix, threshold=0.52)

values

[[('MACD', 'daily_return'), 0.5398124424112505],
 [('force_index', 'SMA_20'), 0.570285049036803],
 [('aroon_oscillator', 'SMA_20'), 0.6399971355369165],
 [('force_index', 'EMA_20'), 0.6004149037747021],
 [('MACD', 'RSI_20'), 0.627834696116851],
 [('EMA_20', 'Stochastic_Oscillator_6_20'), 0.7518276044896368],
 [('aroon_oscillator', 'CMF_20'), 0.5978303868281217],
 [('EMA_20', 'CCI_20'), 0.6008724229326867],
 [('force_index', 'CCI_20'), 0.6985260666209748]]

In [7]:
data = retrieve_data_and_add_features(ticker, scraper, start='2010-01-01', end='2024-01-01')

/Users/jeremylevitt/IdeaProjects/Markov_Structured/venv/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed

In [10]:
data[30:]

,Open,High,Low,Close,Adj Close,Volume,daily_return,volume_change,STD_L=20,SMA_20,...,Bollinger_Upper_Band,Bollinger_Lower_Band,CMF_20,CCI_20,volume_oscillator,force_index,on_balance_volume,aroon_oscillator,volume_price_trend,ultimate_oscillator
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-17,110.269997,110.410004,109.739998,110.260002,84.507103,168845100,0.004738,0.059803,0.012910,-0.002049,...,112.108292,105.724707,-0.012256,53.375075,1.914234e+07,-1.193531e+07,-46269600,-60.0,-1.893189e+07,NaN
2010-02-18,110.080002,111.139999,110.040001,110.910004,85.005302,193708600,0.005895,0.147256,0.012877,-0.001245,...,111.436455,106.098545,-0.000698,94.181072,1.627035e+07,2.595012e+07,147439000,-60.0,-1.778994e+07,NaN
2010-02-19,110.620003,111.570000,110.360001,111.139999,85.181572,222684900,0.002074,0.149587,0.012173,-0.000180,...,111.326347,106.152653,0.071905,119.898082,1.451735e+07,5.756453e+06,370123900,-60.0,-1.732816e+07,NaN
2010-02-22,111.550003,111.580002,110.830002,111.160004,85.196930,132346900,0.000180,-0.405676,0.011006,0.000943,...,111.571761,106.102239,0.135594,120.526444,1.228569e+07,-1.600549e+07,502470800,-60.0,-1.730433e+07,NaN
2010-02-23,110.860001,111.199997,109.519997,109.809998,84.162224,207497000,-0.012145,0.567827,0.011333,0.000080,...,111.575773,106.102228,0.118167,68.512925,-3.679128e+06,-3.024558e+07,294973800,-52.0,-1.982433e+07,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,473.859985,475.380005,471.700012,473.649994,472.182892,67126600,0.002010,-0.225470,0.005951,0.002197,...,475.761452,450.773551,0.274906,105.691976,1.060704e+07,8.035626e+07,2588278200,92.0,-2.849865e+08,NaN
2023-12-26,474.070007,476.579987,473.989990,475.649994,474.176697,55387000,0.004223,-0.174887,0.005891,0.002498,...,477.126709,451.525293,0.297755,108.597303,1.007347e+07,9.866112e+07,2643665200,100.0,-2.847527e+08,NaN
2023-12-27,475.440002,476.660004,474.890015,476.510010,475.034058,68000300,0.001808,0.227730,0.005882,0.002539,...,478.412204,452.397800,0.330973,102.277773,9.398792e+06,8.527706e+07,2711665500,100.0,-2.846297e+08,NaN
